**Data Processing Stage 2** 

In [5]:
!gdown --id 1eHgcyfH1JrKoLxo-BB3D5OZrKxq39-tq
!unzip "/kaggle/working/datafortrainingmodel.zip" -d /kaggle/working/

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1eHgcyfH1JrKoLxo-BB3D5OZrKxq39-tq
From (redirected): https://drive.google.com/uc?id=1eHgcyfH1JrKoLxo-BB3D5OZrKxq39-tq&confirm=t&uuid=b0dce71d-e9d0-4af3-b57a-825997016be5
To: /kaggle/working/datafortrainingmodel.zip
100%|██████████████████████████████████████| 1.77G/1.77G [00:24<00:00, 71.1MB/s]
Archive:  /kaggle/working/datafortrainingmodel.zip
  inflating: /kaggle/working/part-00184-99fe0d07-b76f-4403-81aa-16febd981e1f-c000.csv.gz  
  inflating: /kaggle/working/part-00399-99fe0d07-b76f-4403-81aa-16febd981e1f-c000.csv.gz  
  inflating: /kaggle/working/part-00383-99fe0d07-b76f-4403-81aa-16febd981e1f-c000.csv.gz  
  inflating: /kaggle/working/part-00794-99fe0d07-b76f-4403-81aa-16febd981e1f-c000.csv.gz  
  inf

**Add in Urbanization Rate & isfreegood variables**

In [6]:
!gdown --id 1fIUU7JRLFRVXD9zCFIOUJ3CyYZmVSZ-a
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col,to_date,when

urbandata= spark.read.csv('/kaggle/working/Urbanization Rate Master 2023.csv')
header = urbandata.first()
# Remove the first row from the DataFrame
urbandata = urbandata.filter(col("`_c0`") != header._c0) # Assuming _c0 is not null in the header
# Create a list of column names from the header row
columns = [header[i] for i in range(len(header))]
# Rename columns in the DataFrame
urbandata = urbandata.select([col(f"`_c{i}`").alias(name) for i, name in enumerate(columns)])
urbandata = urbandata.withColumn('% urbanization rate', col('% urbanization rate').cast(DoubleType()))
urbandata.show(2)
urbandata.printSchema()


/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1fIUU7JRLFRVXD9zCFIOUJ3CyYZmVSZ-a
To: /kaggle/working/Urbanization Rate Master 2023.csv
100%|██████████████████████████████████████| 69.5M/69.5M [00:01<00:00, 59.5MB/s]
+----------+--------+-----------------+-------------------+
|customerid|    city|Province standard|% urbanization rate|
+----------+--------+-----------------+-------------------+
|0070246174| CẦN THƠ|          Can Tho|               70.5|
|0070295065|HÀ NỘI|           Ha Noi|              49.05|
+----------+--------+-----------------+-------------------+
only showing top 2 rows

root
 |-- customerid: string (nullable = true)
 |-- city: string (nullable = true)
 |-- Province standard: string (nullable = true)
 |-- % urbanization rate: double (nullabl

In [7]:
import zipfile
import os
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("CSV Reader").getOrCreate()

# Path to the zip file
zip_path = "/kaggle/working/datafortrainingmodel.zip"
extract_path = "/kaggle/working/extracted_files"

# Step 1: Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Step 2: Read all CSV.gz files into one DataFrame
csv_files = [os.path.join(extract_path, f) for f in os.listdir(extract_path) if f.endswith('.csv.gz')]

# Read the CSV.gz files into a single DataFrame
filtered_outliers_df = spark.read.option("header", True).csv(csv_files)

# Show the DataFrame (for confirmation)

from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col,to_date,when

filtered_outliers_df = filtered_outliers_df.withColumn(
    "Ontime", when(col("Ontime") == "Yes",1).otherwise(0))
filtered_outliers_df = filtered_outliers_df.withColumn(
    "Infull", when(col("Infull") == "Yes",1).otherwise(0))
filtered_outliers_df = filtered_outliers_df.withColumn(
    "DIFOT", when(col("DIFOT") == "Yes", 1).otherwise(0))

filtered_outliers_df = filtered_outliers_df.withColumn('order_date', to_date(col('order_date'), 'yyyy-MM-dd')) # Định dạng mẫu ví dụ
filtered_outliers_df = filtered_outliers_df.withColumn('settlement_date', to_date(col('settlement_date'), 'yyyy-MM-dd')) # Định dạng mẫu ví dụ
filtered_outliers_df= filtered_outliers_df.withColumn('totalec_sr', col('totalec_sr').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('totalNSR_sr', col('totalNSR_sr').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('totalNSR_so', col('totalNSR_so').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('totalec_so', col('totalec_so').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('diff_ec', col('diff_ec').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('diff_NSR', col('diff_NSR').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('distance_km', col('distance_km').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('DeliveryDay', col('DeliveryDay').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('days_off_count', col('days_off_count').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('Dayprocessing', col('Dayprocessing').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('Ontime', col('Ontime').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('Infull', col('Infull').cast(DoubleType()))
filtered_outliers_df= filtered_outliers_df.withColumn('DIFOT', col('DIFOT').cast(DoubleType()))

filtered_outliers_df.show(2)
filtered_outliers_df.printSchema()


+--------------------+----------+---------------+-----------------+----------------+-------------+------------+-----------+------------+---------+----------+---------+---------+--------------------+----------+-----------+-----------+----------+-------+--------+--------------+---------------+---------------+----------------+-----------------+-----------+--------------+-------------+------+------+-----+
|           order_key|order_date|settlement_date|           region|distributor_code|customer_code|segmentation|channelname|    order_no|item_code|     brand|pack_type|pack_size|              key_so|totalec_sr|totalNSR_sr|totalNSR_so|totalec_so|diff_ec|diff_NSR|GPScuslatitude|GPScuslongitude|GPSdistlatitude|GPSdistlongitude|      distance_km|DeliveryDay|days_off_count|Dayprocessing|Ontime|Infull|DIFOT|
+--------------------+----------+---------------+-----------------+----------------+-------------+------------+-----------+------------+---------+----------+---------+---------+-------------

In [8]:
#1.Left join to get Urban rate for Factdata 

from pyspark.sql.functions import col

# Join factdata với GPScus, đặt tên lại cho các cột từ GPScus
filtered_outliers_df = filtered_outliers_df.join(urbandata.alias('urbandata'), filtered_outliers_df.customer_code == urbandata.customerid, "left")
# View data frame
filtered_outliers_df = filtered_outliers_df.select(
'order_key',
'order_date',
'settlement_date',
'region',
'distributor_code',
'customer_code',
'segmentation',
'channelname',
'order_no',
'item_code',
'brand',
'pack_type',
'pack_size',
'key_so',
'totalec_sr',
'totalNSR_sr',
'totalNSR_so',
'totalec_so',
'diff_ec',
'diff_NSR',
'GPScuslatitude',
'GPScuslongitude',
'GPSdistlatitude',
'GPSdistlongitude',
'distance_km',
'DeliveryDay',
'days_off_count',
'Dayprocessing',
'Ontime',
'Infull',
'DIFOT',
'city',
'Province standard',
'% urbanization rate'
)

filtered_outliers_df.printSchema()
filtered_outliers_df.show(2)



root
 |-- order_key: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- settlement_date: date (nullable = true)
 |-- region: string (nullable = true)
 |-- distributor_code: string (nullable = true)
 |-- customer_code: string (nullable = true)
 |-- segmentation: string (nullable = true)
 |-- channelname: string (nullable = true)
 |-- order_no: string (nullable = true)
 |-- item_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- pack_type: string (nullable = true)
 |-- pack_size: string (nullable = true)
 |-- key_so: string (nullable = true)
 |-- totalec_sr: double (nullable = true)
 |-- totalNSR_sr: double (nullable = true)
 |-- totalNSR_so: double (nullable = true)
 |-- totalec_so: double (nullable = true)
 |-- diff_ec: double (nullable = true)
 |-- diff_NSR: double (nullable = true)
 |-- GPScuslatitude: string (nullable = true)
 |-- GPScuslongitude: string (nullable = true)
 |-- GPSdistlatitude: string (nullable = true)
 |-- GPSdistlongitude: s

+--------------------+----------+---------------+---------------+----------------+-------------+------------+-----------+------------+---------+------+---------+---------+--------------------+----------+-----------+-----------+----------+-------+--------+--------------+---------------+---------------+----------------+-----------------+-----------+--------------+-------------+------+------+-----+----------+-----------------+-------------------+
|           order_key|order_date|settlement_date|         region|distributor_code|customer_code|segmentation|channelname|    order_no|item_code| brand|pack_type|pack_size|              key_so|totalec_sr|totalNSR_sr|totalNSR_so|totalec_so|diff_ec|diff_NSR|GPScuslatitude|GPScuslongitude|GPSdistlatitude|GPSdistlongitude|      distance_km|DeliveryDay|days_off_count|Dayprocessing|Ontime|Infull|DIFOT|      city|Province standard|% urbanization rate|
+--------------------+----------+---------------+---------------+----------------+-------------+--------

In [9]:
filtered_outliers_df.cache()
filtered_outliers_df=filtered_outliers_df.filter(
    (col("% urbanization rate").isNotNull())
)   
filtered_outliers_df.show(10)

filtered_outliers_df.createOrReplaceTempView("urban_table")

distinct_values_df = spark.sql("SELECT DISTINCT `% urbanization rate` FROM urban_table")

# Check distinct value of urban rate variables
distinct_values = distinct_values_df.collect()

for row in distinct_values:
    print(row["% urbanization rate"])

from pyspark.sql.functions import count

count_per_distinct_value = filtered_outliers_df.groupBy("% urbanization rate").agg(count("*").alias("count"))

# Print result
distinct_counts = count_per_distinct_value.collect()

for row in distinct_counts:
    print(f"city: {row['% urbanization rate']}, Count: {row['count']}")

+--------------------+----------+---------------+---------------+----------------+-------------+------------+-----------+------------+---------+---------+---------+---------+--------------------+----------+-----------+-----------+----------+-------+--------+--------------+---------------+---------------+----------------+-----------------+-----------+--------------+-------------+------+------+-----+----------+-----------------+-------------------+
|           order_key|order_date|settlement_date|         region|distributor_code|customer_code|segmentation|channelname|    order_no|item_code|    brand|pack_type|pack_size|              key_so|totalec_sr|totalNSR_sr|totalNSR_so|totalec_so|diff_ec|diff_NSR|GPScuslatitude|GPScuslongitude|GPSdistlatitude|GPSdistlongitude|      distance_km|DeliveryDay|days_off_count|Dayprocessing|Ontime|Infull|DIFOT|      city|Province standard|% urbanization rate|
+--------------------+----------+---------------+---------------+----------------+-------------+--

51.32
24.31
58.48
27.75
32.51
17.69
20.24
31.65
15.56
38.6
24.24
28.0
32.53
28.78
40.73
23.07
52.81
18.33
44.76
13.88
22.3
84.32
77.77
27.66
13.33
49.05
41.19
14.25
13.98
15.95
29.24
16.62
32.76
67.5
33.55
37.24
33.87
21.11
30.5
42.36
21.61
45.58
19.5
25.76
26.5
16.25
25.5
22.74
19.34
32.7
87.45
28.1
15.2
35.5
25.43
20.85
23.17
20.3
11.81
17.51
22.78
70.5
39.29


city: 51.32, Count: 932164
city: 24.31, Count: 122405
city: 58.48, Count: 740520
city: 27.75, Count: 181119
city: 32.51, Count: 77896
city: 17.69, Count: 246719
city: 20.24, Count: 603364
city: 31.65, Count: 554876
city: 15.56, Count: 1480761
city: 38.6, Count: 1271509
city: 24.24, Count: 264530
city: 28.0, Count: 429200
city: 32.53, Count: 519504
city: 28.78, Count: 540508
city: 40.73, Count: 894222
city: 23.07, Count: 258359
city: 52.81, Count: 1024882
city: 18.33, Count: 690434
city: 44.76, Count: 2049536
city: 13.88, Count: 312785
city: 22.3, Count: 872645
city: 84.32, Count: 1013566
city: 77.77, Count: 9531970
city: 27.66, Count: 1217131
city: 13.33, Count: 242838
city: 49.05, Count: 10936194
city: 41.19, Count: 719200
city: 14.25, Count: 739246
city: 13.98, Count: 736974
city: 15.95, Count: 329637
city: 29.24, Count: 747587
city: 16.62, Count: 664241
city: 32.76, Count: 410200
city: 67.5, Count: 1269572
city: 33.55, Count: 1106205
city: 37.24, Count: 482145
city: 33.87, Count: 28

In [10]:
!gdown --id 1NFGxkgaInoRejTzuU6WlJSYAmZdT99OH
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col,to_date,when,concat, col, lit

isfreegood= spark.read.csv('/kaggle/working/Order have free good.csv')
header1 = isfreegood.first()
# Remove the first row from the DataFrame
isfreegood = isfreegood.filter(col("`_c0`") != header1._c0) # Assuming _c0 is not null in the header
# Create a list of column names from the header row
columns = [header1[i] for i in range(len(header1))]
# Rename columns in the DataFrame
isfreegood = isfreegood.select([col(f"`_c{i}`").alias(name) for i, name in enumerate(columns)])
isfreegood = isfreegood.withColumn('order_date', to_date(col('order_date'), 'yyyy-MM-dd')) # Định dạng mẫu ví dụ
isfreegood = isfreegood.withColumn('key',concat(col('order_date').cast("string"), lit('-'), col('order_no'), lit('-'), col('customer_code')))

isfreegood.show(2)
isfreegood.printSchema()



/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1NFGxkgaInoRejTzuU6WlJSYAmZdT99OH
To: /kaggle/working/Order have free good.csv
100%|██████████████████████████████████████| 50.7M/50.7M [00:00<00:00, 72.8MB/s]
+----------------+-------------+----------+--------------------+
|        order_no|customer_code|order_date|                 key|
+----------------+-------------+----------+--------------------+
|    000041587332|   0070050155|2024-03-04|2024-03-04-000041...|
|VN156005ORD-2710|   0071073113|2024-03-08|2024-03-08-VN1560...|
+----------------+-------------+----------+--------------------+
only showing top 2 rows

root
 |-- order_no: string (nullable = true)
 |-- customer_code: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- key: string (null

In [11]:
from pyspark.sql.functions import concat, col, lit

isfreegood_renamed = isfreegood.withColumnRenamed("order_date", "isfreegood_order_date") \
                               .withColumnRenamed("order_no", "isfreegood_order_no") \
                               .withColumnRenamed("customer_code", "isfreegood_customer_code")

filtered_outliers_df = filtered_outliers_df.withColumn(
    "key",
    concat(
        col("order_date").cast("string"),
        lit('-'),
        col("order_no"),
        lit('-'),
        col("customer_code")
    )
)

isfreegood_renamed = isfreegood_renamed.withColumn(
    "key",
    concat(
        col("isfreegood_order_date").cast("string"),
        lit('-'),
        col("isfreegood_order_no"),
        lit('-'),
        col("isfreegood_customer_code")
    )
)

# Join isfreegood with factdata
filtered_outliers_df = filtered_outliers_df.join(isfreegood_renamed, "key", "left")

# Display dataframe
filtered_outliers_df = filtered_outliers_df.select(
'key',
'order_key',
'order_date',
'settlement_date',
'region',
'distributor_code',
'customer_code',
'segmentation',
'channelname',
'order_no',
'item_code',
'brand',
'pack_type',
'pack_size',
'key_so',
'totalec_sr',
'totalNSR_sr',
'totalNSR_so',
'totalec_so',
'diff_ec',
'diff_NSR',
'GPScuslatitude',
'GPScuslongitude',
'GPSdistlatitude',
'GPSdistlongitude',
'distance_km',
'DeliveryDay',
'days_off_count',
'Dayprocessing',
'Ontime',
'Infull',
'DIFOT',
'city',
'Province standard',
'% urbanization rate',
'isfreegood_order_no'
)
from pyspark.sql import functions as F

# Fill NA 'isfreegood_order_no'
filtered_outliers_df = filtered_outliers_df.withColumn(
    'isfreegood_order_no',
    F.when(F.col('isfreegood_order_no').isNull(), 0).otherwise(1)
)
filtered_outliers_df = filtered_outliers_df.withColumn('isfreegood_order_no', col('isfreegood_order_no').cast(DoubleType())) # Định dạng mẫu ví dụ
# Show result
filtered_outliers_df.select("isfreegood_order_no").describe().show()

filtered_outliers_df.show(2)

filtered_outliers_df.printSchema()

+-------+-------------------+
|summary|isfreegood_order_no|
+-------+-------------------+
|  count|           61702442|
|   mean|0.09564726789905657|
| stddev|0.29410690138854956|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



+--------------------+--------------------+----------+---------------+-----------------+----------------+-------------+------------+-----------+------------+---------+---------+---------+---------+--------------------+----------+-----------+-----------+----------+-------+--------+--------------+---------------+---------------+----------------+-----------------+-----------+--------------+-------------+------+------+-----+---------+-----------------+-------------------+-------------------+
|                 key|           order_key|order_date|settlement_date|           region|distributor_code|customer_code|segmentation|channelname|    order_no|item_code|    brand|pack_type|pack_size|              key_so|totalec_sr|totalNSR_sr|totalNSR_so|totalec_so|diff_ec|diff_NSR|GPScuslatitude|GPScuslongitude|GPSdistlatitude|GPSdistlongitude|      distance_km|DeliveryDay|days_off_count|Dayprocessing|Ontime|Infull|DIFOT|     city|Province standard|% urbanization rate|isfreegood_order_no|
+-------------

**Export Data after Processing**

In [13]:
import zipfile
import os

# Step 1: Cache the DataFrame to improve performance if it's being reused
filtered_outliers_df.cache()

# Step 2: Specify the output path for compressed CSV (GZIP)
output_dir = "/kaggle/working/datafortrainingmodelfinal_csv"
output_gz_file = "/kaggle/working/datafortrainingmodelfinal.csv.gz"

# Step 3: Write the DataFrame to a compressed CSV file using GZIP
filtered_outliers_df.write.csv(output_dir, header=True, mode='overwrite', compression='gzip')

# Step 4: Compress the generated GZIP CSV file into a ZIP file
zip_file_path = "/kaggle/working/datafortrainingmodelfinal.zip"

# Initialize the ZipFile object
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Walk through the output directory and compress only .gz files
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            if file.endswith('.gz'):
                file_path = os.path.join(root, file)
                # Add each .gz file to the zip archive with the original filename
                zipf.write(file_path, os.path.basename(file_path))

# Step 5: Create a download link for the ZIP file (Kaggle/Jupyter environment)
from IPython.display import FileLink, display
display(FileLink(zip_file_path))


/kaggle/working/datafortrainingmodelfinal.zip